In [1]:
from collections import defaultdict
from itertools import *
import json
from operator import *
import pickle
import re

import matplotlib.pylab as pl
import nltk
import numpy as np
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from scipy.sparse import csr_matrix

In [2]:
input_dir = '/data/yu_gai/cfq'
output_dir = '/work/yu_gai/cfq/data/cfq'

df = sqlCtx.read.parquet(f'{input_dir}/dataset.parquet').sort('index').persist()
df.columns

['complexityMeasures',
 'expectedResponse',
 'expectedResponseWithMids',
 'index',
 'question',
 'questionPatternModEntities',
 'questionTemplate',
 'questionWithBrackets',
 'questionWithMids',
 'ruleIds',
 'ruleTree',
 'sparql',
 'sparqlPattern',
 'sparqlPatternModEntities']

In [3]:
splits = {}
split_ids = !ls {input_dir}/splits | grep json
for split_id in [s.replace('.json', '') for s in split_ids]:
    split = splits[split_id] = json.load(open(f'{input_dir}/splits/{split_id}.json'))
    np.savez(f'{output_dir}/splits/{split_id}', **{k : np.array(v) for k, v in split.items()})
    print(split_id, len(split['trainIdxs']), len(split['devIdxs']), len(split['testIdxs']), df.count())

mcd1 95743 11968 11968 239357
mcd2 95743 11968 11968 239357
mcd3 95743 11968 11968 239357
query_complexity_split 100654 9512 9512 239357
query_pattern_split 94600 12489 12589 239357
question_complexity_split 98999 10339 10340 239357
question_pattern_split 95654 12115 11909 239357
random_split 95744 11967 11967 239357


In [4]:
def replace(q):
    for s in [
        'art director',
        'country of nationality',
        'costume designer',
        'executive producer',
        'executive produce',
        'executive produced',
        'film director',
        'film distributor',
        'film editor',
        'film producer',
        'production company',
    ]:
        q = q.replace(s, s.replace(' ', ''))
    return q

df = df.withColumn('questionPatternModEntities', udf(replace, StringType())('questionPatternModEntities')).persist()
df.rdd.map(lambda r: len(r['questionPatternModEntities'].split(' ')) == len(r['questionTemplate'].split(' '))).reduce(and_)

True

In [5]:
at = lambda i: (lambda x: x[i])
k1 = lambda r: [r, 1]
unique = lambda rdd: sorted(rdd.distinct().collect())
count = lambda rdd: dict(rdd.map(k1).reduceByKey(add).collect())
collect = lambda rdd: np.array(rdd.collect())
flat_collect = lambda rdd: np.array(rdd.flatMap(lambda r: r).collect())

In [6]:
SEP, NIL = '{SEP}', '{NIL}'

In [11]:
tok_vocab = idx2tok, tok2idx = pickle.load(open(f'{output_dir}/tok-vocab.pickle', 'rb'))
tag_vocab = idx2tag, tag2idx = pickle.load(open(f'{output_dir}/tag-vocab.pickle', 'rb'))
typ_vocab = idx2typ, typ2idx = pickle.load(open(f'{output_dir}/typ-vocab.pickle', 'rb'))
idx2attr, _ = pickle.load(open(f'{output_dir}/attr-vocab.pickle', 'rb'))
roles, _ = pickle.load(open(f'{output_dir}/role-vocab.pickle', 'rb'))

In [12]:
def find_rel(line):
    if 'FILTER' in line:
        [[src, dst, *_]] = re.findall(r'^FILTER \( ([^ ]+) != ([^ ]+) \)( .)?$', line)
        return src, '!=', dst
    else:
        [[src, typ, dst, *_]] = re.findall(r'^([^ ]+) ([^ ]+) ([^ ]+)( .)?$', line)
        return src, typ, dst

r = '(?:%s)' % '|'.join(fr'\[{role[1 : -1]}\]' for role in roles)  # TODO
# p = re.compile(fr'{r} and {r}|(?:{r} , )+and {r}')
p = re.compile('|'.join(fr'{r} and {r}|(?:{r} , )+and {r}' for r in [fr'\[{role[1 : -1]}\]' for role in roles]))
def grp_by_tag(tags):
    lens = np.array(list(map(len, tags)))
    ends = np.cumsum(lens) + np.arange(len(tags))
    starts = ends - lens

    t = ' '.join(tags)
    homo = lambda s: sum(role in s for role in roles) == 1
    matches = [m for m in re.finditer(p, t) if homo(m.group())]
    if not matches:
        grps = [[i] for i in range(len(tags))]
        return grps
    
    m_start, m_end = zip(*([m.start(), m.end()] for m in matches))
    hit = False
    grps = []
    for idx, [start, end] in enumerate(zip(starts, ends)):
        if start in m_start:
            hit = True
            grps.append([])
        if hit:
            grps[-1].append(idx)
        else:
            grps.append([idx])
        if end in m_end:
            hit = False
    
    for start, end, grp in zip(m_start, m_end, (grp for grp in grps if len(grp) > 1)):
        assert t[start : end] == ' '.join(tags[idx] for idx in grp)

    return grps

In [9]:
def _mapper(r):
    rels = list(map(find_rel, r['sparqlPatternModEntities'].split('\n')[1 : -1]))
    srcs, typs, dsts = zip(*rels)
    ents = sorted({x for x in chain(srcs, dsts) if re.match('M\d', x)})

    toks = r['questionPatternModEntities'].split(' ')
    tags = r['questionTemplate'].split(' ')
    grps = grp_by_tag(tags)

    return ' '.join(tags[idx] for idx, *_ in grps)

df.rdd.map(_mapper).distinct().count()

18378

def _mapper(r):
    rels = list(map(find_rel, r['sparqlPatternModEntities'].split('\n')[1 : -1]))
    srcs, typs, dsts = zip(*rels)
    ents = sorted({x for x in chain(srcs, dsts) if re.match('M\d', x)})

    toks = r['questionPatternModEntities'].split(' ')
    tags = r['questionTemplate'].split(' ')
    grps = grp_by_tag(tags)
    
    templ = [tags[idx].replace('[', '').replace(']', '') for idx, *_ in grps]
    starts, ends, _ = zip(*find_noun_phrases(templ))
    idx, seq = 0, []
    while idx < len(grps):
        if idx in starts:
            seq.append('NP_COMPLEX')
            idx = ends[starts.index(idx)]
        else:
            seq.append(templ[idx])
            idx += 1
    
    return r['index'], ' '.join(seq)

templs = df.rdd.map(_mapper).cache()
uniq_templs1, uniq_templs2, uniq_templs3 = [{k : set(templs.filter(lambda r: r[0] in v).map(at(1)).collect())
                                             for k, v in splits[f'mcd{i}'].items()} for i in [1, 2, 3]]

In [82]:
def _mapper(r):
    dat = {}
    
    rels = list(map(find_rel, r['sparqlPatternModEntities'].split('\n')[1 : -1]))
    srcs, typs, dsts = zip(*rels)
    ents = sorted({x for x in chain(srcs, dsts) if re.match('M\d', x) or re.match('\?x\d', x)})
    seq_var = sorted(ent for ent in ents if ent.startswith('?x'))

    toks = r['questionPatternModEntities'].split(' ') + [NIL] + seq_var
    tags = r['questionTemplate'].split(' ') + [NIL] + seq_var
    toks, tags = zip(*([tok, tag] for tok, tag in zip(toks, tags) if tag != '[ADJECTIVE_SIMPLE]'))
    assert len(toks) == len(tags)
    grps = grp_by_tag(tags)

    dat['seq'] = [tag2idx[tags[idx]] for idx, *_ in grps]
    dat['mem'] = [[tok2idx[toks[grp[0]]]] if len(grp) == 1 else
                  [tok2idx[toks[idx]] for idx in grp if tags[idx] in roles] for grp in grps if not toks[grp[0]].startswith('?x')]
    
    ent2grp = {}
    for idx, tok in zip(chain(*(len(grp) * [idx] for idx, grp in enumerate(grps))), toks):
        if tok in ents:
            ent2grp[tok] = idx
    idx2grp = dat['idx2grp'] = sorted(set(ent2grp.values()))
    ent2idx = {ent : idx2grp.index(ent2grp[ent]) for ent in ents}
    
    dat['rel'] = [[ent2idx[src], ent2idx[dst], idx2typ.index(typ)] for src, typ, dst in rels if typ in idx2typ]

    return dat

dat, rdd = {}, df.rdd.map(_mapper).cache()
dat['n_rel'] = collect(rdd.map(at('rel')).map(len))
rdd_rel = rdd.flatMap(at('rel')).cache()
dat['src'], dat['dst'], dat['typ'] = [collect(rdd_rel.map(at(i))) for i in range(3)]
dat['n_grp'] = collect(rdd.map(at('mem')).map(len))
rdd_mem = rdd.flatMap(at('mem')).cache()
dat['n_mem'], dat['mem'] = collect(rdd_mem.map(len)), flat_collect(rdd_mem)
dat['n'] = collect(rdd.map(at('idx2grp')).map(len))
dat['idx2grp'] = collect(rdd.flatMap(at('idx2grp')))

In [83]:
# grammar = nltk.CFG.fromstring("""
#     S -> NP
#     DT -> 'a' | 'an'
#     JJ -> 'ADJECTIVE_SIMPLE'
#     N -> 'NP_SIMPLE' | 'ROLE_SIMPLE' | JJ N
#     NP -> 'entity' | DT N | 'What' | 'What' N | 'Which' N | 'Who' | NP POS N | NP PP NP | NP 'whose' N
#     POS -> "'s"
#     PP -> 'of'
# """)
grammar = nltk.CFG.fromstring("""
    S -> NP
    DT -> 'a' | 'an'
    JJ -> 'ADJECTIVE_SIMPLE'
    N -> 'NP_SIMPLE' | 'ROLE_SIMPLE' | JJ N
    NP -> 'entity' | DT N | 'What' | 'What' N | 'Which' N | 'Who' | NP POS N | NP PP NP | NP 'whose' N
    POS -> "'s"
    PP -> 'of'
""")
parser = nltk.ChartParser(grammar)

def find_noun_phrases(tags):
    hits = []
    start = 0
    while start < len(tags):
        for end in range(len(tags), start, -1):
            try:
                trees = list(parser.parse(tags[start : end]))
            except ValueError:
                trees = []
            if len(trees) > 0:
                hits.append([start, end, trees])
                start = end - 1
                break
        start += 1
    return hits
            
for start, end, trees in find_noun_phrases("a ADJECTIVE_SIMPLE NP_SIMPLE of entity".split(' ')):
    print(start, end)
    for tree in trees:
        tree.pretty_print()

0 5
                             S               
                             |                
                             NP              
            _________________|___________     
           NP                       |    |   
  _________|__________              |    |    
 |                    N             |    |   
 |          __________|______       |    |    
 DT        JJ                N      PP   NP  
 |         |                 |      |    |    
 a  ADJECTIVE_SIMPLE     NP_SIMPLE  of entity



In [84]:
def _mapper(r):
    dat = {}
    
    idx2tag_ = [tag.replace('[', '').replace(']', '') for tag in idx2tag]
    templ = [idx2tag_[idx] for idx in takewhile(lambda idx: not idx2tag[idx].startswith('?x'), r['seq'])]
    hits = dat['hits'] = find_noun_phrases(templ)

    dat['seq_noun'] = [r['seq'][start : end] for start, end, _ in hits]
    
    idx = 0
    starts, ends, trees = zip(*hits)
    seq_tag = dat['seq_tag'] = []
    pos_np = dat['pos_np'] = []
    pos_all = dat['pos_all'] = []
    istag, isnoun, isvar = dat['istag'], dat['isnoun'], dat['isvar'] = [], [], []
    while idx < len(templ):
        if idx in starts:
            start, end, _ = hits[starts.index(idx)]
            disp = end - start

            pos_np.append(len(seq_tag))
            seq_tag.append(len(idx2tag_))
            
            pos_all.extend(range(disp))
            istag.extend(disp * [False])
            isnoun.extend(disp * [True])
            isvar.extend(disp * [False])
            
            idx = end
        else:
            seq_tag.append(idx2tag_.index(templ[idx]))
            
            pos_all.append(len(seq_tag) - 1)
            istag.append(True)
            isnoun.append(False)
            isvar.append(False)
            
            idx += 1
            
    seq_var = dat['seq_var'] = r['seq'][len(templ):] + [idx2tag.index('{NIL}')]   
    pos_all.extend(range(len(seq_var)))
    istag.extend(len(seq_var) * [False])
    isnoun.extend(len(seq_var) * [False])
    isvar.extend(len(seq_var) * [True])

    return dat

rdd_np = rdd.map(_mapper).cache()
dat['len_tag'] = collect(rdd_np.map(at('seq_tag')).map(len))
dat['seq_tag'] = collect(rdd_np.flatMap(at('seq_tag')))
rdd_noun = rdd_np.flatMap(at('seq_noun')).cache()
dat['len_noun'] = collect(rdd_noun.map(len))
dat['seq_noun'] = flat_collect(rdd_noun)
dat['len_var'] = collect(rdd_np.map(at('seq_var')).map(len))
dat['seq_var'] = flat_collect(rdd_np.map(at('seq_var')))
dat['n_np'] = collect(rdd_np.map(at('pos_np')).map(len))
dat['pos_np'] = collect(rdd_np.flatMap(at('pos_np')))
dat['n_all'] = collect(rdd_np.map(at('pos_all')).map(len))
dat['pos_all'] = collect(rdd_np.flatMap(at('pos_all')))
dat['istag'] = collect(rdd_np.flatMap(at('istag')))
dat['isnoun'] = collect(rdd_np.flatMap(at('isnoun')))
dat['isvar'] = collect(rdd_np.flatMap(at('isvar')))
np.savez(f'{output_dir}/data', **dat)

In [85]:
uniq_lhs = set(str(p.lhs()) for p in grammar.productions())
uniq_rhs = set(chain(*(map(str, p.rhs()) for p in grammar.productions())))
idx2symb = sorted(uniq_lhs.union(uniq_rhs))
symb2idx = {symb : idx for idx, symb in enumerate(idx2symb)}
pickle.dump([idx2symb, symb2idx], open(f'{output_dir}/symb-vocab.pickle', 'wb'))
idx2symb, symb2idx

(["'s",
  'ADJECTIVE_SIMPLE',
  'DT',
  'JJ',
  'N',
  'NP',
  'NP_SIMPLE',
  'POS',
  'PP',
  'ROLE_SIMPLE',
  'S',
  'What',
  'Which',
  'Who',
  'a',
  'an',
  'entity',
  'of',
  'whose'],
 {"'s": 0,
  'ADJECTIVE_SIMPLE': 1,
  'DT': 2,
  'JJ': 3,
  'N': 4,
  'NP': 5,
  'NP_SIMPLE': 6,
  'POS': 7,
  'PP': 8,
  'ROLE_SIMPLE': 9,
  'S': 10,
  'What': 11,
  'Which': 12,
  'Who': 13,
  'a': 14,
  'an': 15,
  'entity': 16,
  'of': 17,
  'whose': 18})

In [86]:
def traverse(tree, nid=0):
    labels, isleaf, edges = [tree.label()], [False], []
    subtree_nid = nid + 1
    for subtree in tree:
        edges.append([nid, subtree_nid])
        if isinstance(subtree, nltk.tree.Tree):
            subtree_labels, subtree_isleaf, subtree_edges = traverse(subtree, subtree_nid)
            subtree_nid += len(subtree_labels)
            labels.extend(subtree_labels)
            isleaf.extend(subtree_isleaf)
            edges.extend(subtree_edges)
        elif isinstance(subtree, str):
            subtree_nid += 1
            labels.append(subtree)
            isleaf.append(True)
        else:
            raise TypeError()
            
    return labels, isleaf, edges

def _mapper(r):
    dat = {}
    
    dat['symb'], dat['isleaf'], dat['src'], dat['dst'] = [], [], [], []
    for _, _, trees in r['hits']:
        labels, isleaf, edges = traverse(trees[0])
        dat['symb'].append([symb2idx[symb] for symb in labels])
        dat['isleaf'].append(isleaf)
        src, dst = zip(*edges)
        dat['src'].append(src)
        dat['dst'].append(dst)
        
    return dat

rdd_tree = rdd_np.map(_mapper).cache()
dat['n_tree'] = collect(rdd_tree.flatMap(at('symb')).map(len))
dat['symb'] = flat_collect(rdd_tree.flatMap(at('symb')))
dat['isleaf'] = flat_collect(rdd_tree.flatMap(at('isleaf')))
dat['m_tree'] = collect(rdd_tree.flatMap(at('src')).map(len))
dat['src_tree'] = flat_collect(rdd_tree.flatMap(at('src')))
dat['dst_tree'] = flat_collect(rdd_tree.flatMap(at('dst')))
np.savez(f'{output_dir}/data', **dat)

In [ ]:
seq2templ = lambda seq: ' '.join(idx2tag[idx] for idx in takewhile(lambda idx: idx2tag[idx] != '{SEP}', seq))
def _mapper(r):
    tags = seq2templ(r).replace('[', '').replace(']', '').split(' ')
    hits = find_noun_phrases(tags)
    return all(any(start <= idx < end for start, end, _ in hits)
               for idx, tag in enumerate(tags) if tag in ['entity', 'NP_SIMPLE', 'ROLE_SIMPLE'])

rdd.map(at(3)).map(_mapper).reduce(add)

In [ ]:
def _mapper(r):
    tags = seq2templ(r).replace('[', '').replace(']', '').split(' ')
    return [tags[start : end] for start, end, _ in find_noun_phrases(tags)]

rdd_tmp = rdd.map(_mapper).zip(df.rdd.map(lambda r: r['index'])).cache()
rdd_tmp.filter(lambda r: r[1] in splits['mcd3']['trainIdxs']).map(at(0))

In [66]:
idx2tag_ = idx2tag + ['[NP]']
idx2tag_[idx2tag_.index('[VP_SIMPLE]')] = '[VP]'
to_str = lambda templ: ' '.join(idx2tag_[idx] for idx in templ)
templs = [[to_str(templ[:-1]), n] for templ, n in count(rdd_np.map(at('seq_tag')).map(tuple)).items()]

In [69]:
phrases = set(chain(*(templ.split(' , ')[1 : -1] for templ, _ in templs if templ.count(',') > 1)))
phrases

{'[NP] [VP]',
 '[NP] [VP] [NP] [VP]',
 '[NP] that [NP] [VP] [VP]',
 '[NP] that [NP] was [VP] by [VP]',
 '[NP] was [VP] by',
 '[VP]',
 '[VP] [NP]',
 '[VP] [NP] [VP] [NP]',
 '[VP] [NP] [VP] by [NP]',
 '[VP] [NP] that [NP] was [VP] by',
 '[VP] [NP] that [NP] were [VP] by',
 '[VP] [NP] that [VP] [NP]',
 '[VP] by',
 '[VP] by [NP]',
 '[VP] by [NP] [VP] [NP]',
 '[VP] by [NP] [VP] by [NP]',
 '[VP] by [NP] that [NP] was [VP] by',
 '[VP] by [NP] that [NP] were [VP] by',
 '[VP] by [NP] that [VP] [NP]',
 'was [VP] by',
 'was [VP] by [NP]',
 'was [VP] by [NP] that [NP] [VP]',
 'was [VP] by [NP] that [VP] [NP]',
 'were [VP] by'}

In [ ]:
def _mapper(r):
    dat = {}
    
    seq_tag = to_str(r['seq_tag'])
    dat['seq_phr'] = 

In [67]:
templs

[['Did [NP] [VP] [NP] that was [VP] by and [VP] by [NP] and [VP] [NP]', 1],
 ['Did [NP] [VP] by [NP] [VP] [NP]', 34],
 ['Did [NP] that [NP] was [VP] by and was [VP] by [VP] [NP]', 14],
 ['Were [NP] [VP] by [NP] and [VP] by [NP] [VP] [NP]', 14],
 ['Were [NP] [VP] by [NP] that [NP] was [VP] by , [VP] by , [VP] by , and [VP] by',
  1],
 ['[NP] did [NP] that [NP] [VP] and [NP] were [VP] by [VP]', 4],
 ['[NP] that [NP] [VP] and were [VP] by was [VP] by [NP] and [VP] by [NP]', 2],
 ['[NP] that was [VP] by and was [VP] by [NP] was [VP] by and was [VP] by [NP]',
  1],
 ['[NP] was [VP] by [NP] was [VP] by and [VP] by [NP]', 4],
 ['[NP] was [VP] by [NP] was [VP] by , [VP] by , and [VP] by [NP]', 2],
 ['[NP] was [VP] by , [VP] by , and [VP] by [NP] that [NP] were [VP] by', 3],
 ['[NP] was [VP] by [NP] was [VP] by [NP] , [VP] by [NP] , and [VP] by [NP]',
  1],
 ['Did [NP] that [NP] was [VP] by and [NP] [VP] [VP] [NP]', 14],
 ['Was [NP] that [NP] were [VP] by and were [VP] by [VP] by [NP]', 1],
 ['

In [74]:
pred = lambda templ: all(any(part.startswith(phr) for phr in phrases) for part in templ.split(' and ')[1:])
sum(1 for templ, _ in templs if 'and' in templ and pred(templ))

1681

In [75]:
len(templs), sum(1 for templ, _ in templs if 'and' in templ)

(1910, 1718)

In [76]:
[templ for templ, _ in templs if not pred(templ)]

['[NP] did [NP] that [NP] [VP] and [NP] were [VP] by [VP]',
 '[NP] that [NP] [VP] and [NP] were [VP] by [VP] [NP]',
 'Was [NP] [NP] that [NP] [VP] and [NP] that [VP] [NP] was [VP] by',
 '[NP] did [NP] [VP] and [NP] that [VP] and was [VP] by [NP] [VP]',
 'Was [NP] that [NP] were [VP] by and [NP] were [VP] by [NP]',
 '[NP] did [NP] [VP] and [NP] that [NP] were [VP] by [VP]',
 'Did [NP] that [NP] were [VP] by and [NP] were [VP] by [VP] [NP]',
 '[NP] did [NP] [VP] and [NP] that [NP] was [VP] by and [VP] by [VP]',
 'Was [NP] [NP] that [NP] [VP] and [NP] were [VP] by',
 '[NP] that [NP] were [VP] by and [NP] were [VP] by was [NP]',
 '[NP] that [NP] were [VP] by and [NP] were [VP] by did [NP] [VP]',
 'Was [NP] [VP] by [NP] that [NP] were [VP] by and [NP] were [VP] by',
 '[NP] that [NP] [VP] and [NP] were [VP] by was [VP] by [NP]',
 'Was [NP] that [NP] were [VP] by and [NP] were [VP] by [VP] by [NP]',
 '[NP] did [NP] [VP] and [NP] that [VP] [NP] [VP]',
 '[NP] did [NP] [VP] and [NP] that [NP] [V

In [73]:
sum(n for templ, n in templs if not pred(templ))

4969